In [10]:
# load binary file
import numpy as np
import time
import suite2p, register, dcnv, celldetect
import imp
imp.reload(suite2p)
imp.reload(register)

def tic():
    return time.time()
def toc(i0):
    return time.time() - i0

ops = {
    'diameter': 12,
    'tau': 2.,
    'fs': 2.5,
    #'data_path': 'C:/DATA/',
    'data_path': 'C:\Drive\suite2python\data',
    'subfolders': ('4'),
    'num_workers': 0
      }

# copy tiff to a binary
i0 = tic()
ops1 = suite2p.main(ops)
print(toc(i0))
#ops1 = register.tiff_to_binary(ops)

cells: 401, cost: 0.5904, time: 53.6261
cells: 741, cost: 0.5797, time: 116.2852
cells: 1055, cost: 0.5735, time: 183.7259
cells: 1280, cost: 0.5700, time: 256.6356
cells: 1401, cost: 0.5682, time: 329.6087
cells: 1469, cost: 0.5673, time: 401.1963
cells: 1502, cost: 0.5669, time: 482.7552
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
1795.6755194664001


In [ ]:
import imp
imp.reload(register)

i0 = tic()
#ops1[0]['batch_size'] = 200
ops1[0]['num_workers'] = 0

# register tiff
ops1 = register.register_binary(ops1)

print(toc(i0))

In [4]:
import imp
imp.reload(celldetect)

ops = ops1[0]
U,sdmov      = celldetect.getSVDdata(ops)
S, StU , StS = celldetect.getStU(ops, U)
print(U.shape)

(1000, 502, 503)

In [8]:
ops['diameter'] = 12
# get ROIs
ops, stat, cell_masks, neuropil_masks, mPix, mLam = celldetect.sourcery(ops, U, S, StU, StS)
# extract fluorescence and neuropil
F, Fneu = celldetect.extractF(ops, stat, cell_masks, neuropil_masks, mPix, mLam)
# deconvolve fluorescence
spks = dcnv.oasis(F - ops['neucoeff'] * Fneu, ops)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
%matplotlib inline
import scipy.stats as stats

dF = F - 0.7*Fneu
dF = stats.zscore(dF, axis=1)

sk = stats.skew(dF, axis=1)
sd = np.std(dF, axis=1)
print(np.mean(sk[sd>0]))

plt.figure(figsize=(18,18))
plt.imshow(dF[:,:], vmax=5, aspect='auto')
plt.show()
